<a href="https://colab.research.google.com/github/lulezo/Parkinson-s-Disease-Telemonitoring-Dataset-Regression-Model/blob/main/PD_dataset_MLP_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Load the data

In [ ]:
!wget https://archive.ics.uci.edu/static/public/189/data.csv

In [ ]:
import pandas as pd
df = pd.read_csv('data.csv')
df = df.drop('motor_UPDRS', axis=1)
df = df.drop('subject#', axis=1)
df.head()

2. Grid Search and Model Creation

In [3]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

X = df.drop('total_UPDRS', axis=1)  # Features
y = df['total_UPDRS']  # Target variable
n_inputs = X.shape[1]

# split into train test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

# scale data
t = MinMaxScaler()
t.fit(X_train)
X_train = t.transform(X_train)
X_test = t.transform(X_test)

mlp = MLPRegressor(max_iter=100)

parameter_space = {
    'hidden_layer_sizes': [(100, 200, 300, 100), (200, 400, 600, 400, 200, 100)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
    }

reg = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
reg.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


GridSearchCV(cv=3, estimator=MLPRegressor(max_iter=100), n_jobs=-1,
             param_grid={'activation': ['tanh', 'relu'],
                         'alpha': [0.0001, 0.05],
                         'hidden_layer_sizes': [(100, 200, 300, 100),
                                                (200, 400, 600, 400, 200, 100)],
                         'learning_rate': ['constant', 'adaptive'],
                         'solver': ['sgd', 'adam']})

3. Best Parameters

In [4]:
# Best paramete set
print('Best parameters found:\n', reg.best_params_)

# All results
means = reg.cv_results_['mean_test_score']
stds = reg.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, reg.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))


Best parameters found:
 {'activation': 'relu', 'alpha': 0.05, 'hidden_layer_sizes': (200, 400, 600, 400, 200, 100), 'learning_rate': 'adaptive', 'solver': 'sgd'}
0.284 (+/-0.408) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (100, 200, 300, 100), 'learning_rate': 'constant', 'solver': 'sgd'}
-0.000 (+/-0.001) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (100, 200, 300, 100), 'learning_rate': 'constant', 'solver': 'adam'}
0.459 (+/-0.116) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (100, 200, 300, 100), 'learning_rate': 'adaptive', 'solver': 'sgd'}
-0.000 (+/-0.001) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (100, 200, 300, 100), 'learning_rate': 'adaptive', 'solver': 'adam'}
-0.000 (+/-0.001) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (200, 400, 600, 400, 200, 100), 'learning_rate': 'constant', 'solver': 'sgd'}
-0.000 (+/-0.000) for {'activation': 'tanh', 'alpha': 0.0001, 'hidde

4. Results

In [7]:
y_true, y_pred = y_test , reg.predict(X_test)

from sklearn.metrics import mean_absolute_error

# calculate error
score = mean_absolute_error(y_test, y_pred)
print(score)
print(y_test.values)
y_pred

5.775445068700374
[26.23  26.23  20.867 ... 22.644 47.561 38.353]


array([24.42537046, 28.44294639, 31.47598568, ..., 22.60836689,
       49.67788158, 44.63194897])